In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import PolynomialFeatures
from modelo_cuadratico import modelo_cuadratico

### Funcioncitas

In [ ]:
def position_of_item(itemid:int, matrix:pd.DataFrame):
    """
    Busca la posicion de un itemid en las columnas de la matriz de interacciones
    """
    try:
        pos = np.where(matrix.columns==itemid)[0].item()
    except:
        pos = None
    return pos

def position_of_user(userid:int, matrix:pd.DataFrame):
    """
    Busca la posicion de un userid en las filas de la matriz de interacciones
    """
    try:
        pos = np.where(matrix.index==userid)[0].item()
    except:
        pos = None
    return pos

In [ ]:
def itemid_of_position(position:int, matrix:pd.DataFrame):
    """
    Busca el itemid de la posicion dada en la matriz de interacciones
    """
    try:
        itemid = matrix.columns[position]
    except:
        itemid = None
    return itemid

def userid_of_position(position:int, matrix:pd.DataFrame):
    """
    Busca el userid de la posicion dada en la matriz de interacciones
    """
    try:
        userid = matrix.index[position]
    except:
        userid = None
    return userid

### Proto Recomendador
Esta notebook está basada en el paper A Recommendation Model Based on Deep Neural Network del autor LIBO ZHANG

#### Prueba con datos creados

In [ ]:
n_users = 50
n_items = 100
# TODO: Si a y b no son iguales se rompe. Ver por qué.
a = 20
b = 20
sparce_rate = .9 # Qué tan dispersa es la matriz de ratings de ejemplo

# Inicializo la matriz de interacciones y pongo -1 en muchos lugares
# El -1 representa las interacciones que no conocemos o no sucedieron. Ej: las peliculas que no vio el usuario.
ratings = np.random.randint(0, 2, n_users*n_items).reshape(n_users,n_items)
aux = np.random.rand(ratings.shape[0], ratings.shape[1])
ratings[aux<sparce_rate] = -1
ratings = torch.from_numpy(ratings)

In [ ]:
model = modelo_cuadratico(a, b, n_users, n_items)

In [ ]:
%time R = model.entrenar(ratings)

#### Prueba con datos de MovieLens

Uso el dataset de MovieLens 1M (https://grouplens.org/datasets/movielens/1m/).
Para correr esta notebook hay que bajarlo, descomprimirlo y apuntar los siguientes paths a los archivos correspondientes

In [ ]:
user_data = '../../MovieLens/data/users.dat'
movie_data = '../../MovieLens/data/movies.dat'
ratings_data = '../../MovieLens/data/ratings.dat'

In [ ]:
users = pd.read_csv(user_data, sep = '::', header = None, names = ['UserId', 'Gender', 'AgeGroup', 'Occupation', 'Zip'], encoding = 'latin-1')
movies = pd.read_csv(movie_data, sep = '::', header = None, names = ['MovieId', 'Title', 'Genre'], encoding = 'latin-1')
ratings_data = pd.read_csv(ratings_data, sep = '::', header = None, names = ['UserId', 'MovieId', 'Rating', 'Timestamp'], encoding = 'latin-1')

In [ ]:
ratings_data.head()

In [ ]:
ratings_data['Match'] = (ratings_data['Rating']>=4).astype(int) # Considero un match si el ratings es 4 o más (esto es bastante burdo pero es para probar)

In [ ]:
ratings_df = ratings_data.pivot(index='UserId',
                                 columns='MovieId',
                                 values='Match').fillna(-1)

In [ ]:
ratings_df.head()

In [ ]:
ratings_matrix = ratings_df.to_numpy()

In [ ]:
n_users = ratings_data.UserId.nunique()
n_items = ratings_data.MovieId.nunique()
a = 20
b = 20
l = a + b
ratings = ratings_matrix.copy()
ratings = torch.from_numpy(ratings)

In [ ]:
model = modelo_cuadratico(a, b, n_users, n_items)

### Entrenamiento Regresión Cuadrática

In [ ]:
%time R = model.entrenar(ratings, lr = 0.1)